# 前置动作
1. 克隆https://github.com/unit-mesh/unit-minions项目
2. 将userstory_detail_double_clean_cn.jsonl数据集放到input0或者修改下面的数据集路径
3. 将ChatGLM-6b的模型文件准备到input2或者修改下面模型引用的路径。或者直接指定为THUDM/chatglm-6b
4. 如果不是用的openbayes平台，记得修改下方的模型输出路径，并提前创建好路径

In [1]:
!git clone https://github.com/THUDM/ChatGLM-6B.git

Cloning into 'ChatGLM-6B'...
remote: Enumerating objects: 742, done.
remote: Counting objects: 100% (452/452), done.
remote: Compressing objects: 100% (212/212), done.
remote: Total 742 (delta 282), reused 291 (delta 239), pack-reused 290
Receiving objects: 100% (742/742), 5.68 MiB | 16.29 MiB/s, done.
Resolving deltas: 100% (422/422), done.


In [2]:
%cd ChatGLM-6B

/output/ChatGLM-6B


In [3]:
!git pull

Already up to date.


In [4]:
!pip install -r requirements.txt
!pip install rouge_chinese nltk jieba datasets icetk accelerate

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 5.8 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.6/416.6 kB 5.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 3.2 MB/s eta 0:00:0000:0100:04
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 5.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.9 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 5.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.9/771.9 kB 9.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 4.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 4.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.

# 训练

In [3]:
%cd ptuning

/output/ChatGLM-6B/ptuning


In [ ]:
!mkdir -p /output/train
!sed -n '10,15p' /openbayes/input/input0/datasets/userstory/userstory_detail_double_clean_cn.jsonl > /output/train/dev.json
!cp /openbayes/input/input0/datasets/userstory/userstory_detail_double_clean_cn.jsonl /output/train/train.json
!export CUDA_VISIBLE_DEVICES=0 && python main.py \
    --do_train \
    --train_file /output/train/train.json \
    --validation_file /output/train/dev.json \
    --prompt_column input \
    --response_column output \
    --overwrite_cache \
    --model_name_or_path /openbayes/input/input1 \
    --output_dir /output/ptuning/chatglm-6b-pt-128-2e-2 \
    --overwrite_output_dir \
    --max_source_length 512 \
    --max_target_length 512 \
    --per_device_train_batch_size 8 \
    --per_device_eval_batch_size 8 \
    --gradient_accumulation_steps 2 \
    --predict_with_generate \
    --max_steps 3000 \
    --logging_steps 10 \
    --save_steps 1000 \
    --learning_rate 2e-2 \
    --pre_seq_len 128

04/13/2023 08:46:55 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
04/13/2023 08:46:55 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
generation_max_len

# 训练后推理

In [5]:
import os
import torch
from transformers import AutoConfig, AutoModel, AutoTokenizer

# 载入Tokenizer
tokenizer = AutoTokenizer.from_pretrained("/openbayes/input/input1", trust_remote_code=True)
config = AutoConfig.from_pretrained("/openbayes/input/input1", trust_remote_code=True, pre_seq_len=128)
model = AutoModel.from_pretrained("/openbayes/input/input1", config=config, trust_remote_code=True)
prefix_state_dict = torch.load(os.path.join("/output/ptuning/chatglm-6b-pt-128-2e-2/checkpoint-3000", "pytorch_model.bin"))
new_prefix_state_dict = {}
for k, v in prefix_state_dict.items():
    if k.startswith("transformer.prefix_encoder."):
        new_prefix_state_dict[k[len("transformer.prefix_encoder."):]] = v
model.transformer.prefix_encoder.load_state_dict(new_prefix_state_dict)
model = model.half().cuda()
model.transformer.prefix_encoder.float()
model = model.eval()

def evaluate(prompt):
    response, history = model.chat(tokenizer, prompt, history=[])
    print(prompt)
    print(response)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Some weights of ChatGLMForConditionalGeneration were not initialized from the model checkpoint at /openbayes/input/input1 and are newly initialized: ['transformer.prefix_encoder.embedding.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
evaluate("京西商城:浏览最新电影")

京西商城:浏览最新电影
用户故事：可以浏览最新电影
作为一个京西商城的用户
我想在应用中浏览最新电影
以便于我能够及时观看最新的电影

AC 1: 用户可以在应用中浏览最新电影
假设 用户已经登录了京西商城
当 用户点击浏览最新电影按钮
于是 用户可以看到最新的电影列表，并且可以搜索特定的电影


In [15]:
import json

with open('/output/train/evaluate.jsonl', 'r', encoding='utf-8') as json_file:
    json_list = list(json_file)

for json_str in json_list:
    result = json.loads(json_str)
    evaluate(result["input"])
    print("====================================================\r")

电商平台: 用户可以在平台上搜索、购买和评价各种商品，商家可以在平台上开设店铺、发布商品和管理订单。
用户故事：可以在电商平台上搜索、购买和评价各种商品
作为一个电商平台的用户
我想在平台上搜索、购买和评价各种商品
以便于我能够更好地了解和购买各种商品

AC 1: 用户可以在电商平台上搜索、购买和评价各种商品
假设 用户已经登录电商平台
当 用户搜索到自己想要的商品
于是 用户可以购买该商品，并且可以评价该商品
在线教育: 学生可以通过在线课程和教材学习知识，老师可以在平台上创建课程和监督学生进度。
用户故事：可以通过在线教育平台学习知识
作为一个在线教育平台的用户
我想通过在线教育平台学习知识
以便于我可以通过在线课程和教材学习知识

AC 1: 用户可以通过在线教育平台学习知识
假设 用户已经登录到在线教育平台
当 用户点击学习按钮
于是 用户可以通过在线课程和教材学习知识

AC 2: 用户可以在在线教育平台上创建课程
假设 用户已经登录到在线教育平台
当 用户点击创建课程按钮
于是 用户可以在平台上创建课程，并和监督学生进度
社交媒体: 用户可以在平台上分享动态、发表观点和交流信息，平台可以通过广告等方式盈利。
用户可以使用社交媒体平台分享动态、发表观点和交流信息
作为一个社交媒体用户
我想在平台上分享动态、发表观点和交流信息
以便于我可以与其他用户进行交流，并获得反馈

AC 1: 社交媒体用户可以分享动态、发表观点和交流信息
假设 社交媒体用户已经登录
当 社交媒体用户点击分享动态按钮
于是 社交媒体用户可以将动态分享到平台上

AC 2: 社交媒体用户可以发表观点和交流信息
假设 社交媒体用户已经登录
当 社交媒体用户点击发表观点按钮
于是 社交媒体用户可以发表观点和交流信息
金融科技: 用户可以在应用上进行在线支付、理财和投资，平台可以提供风险评估、信用评分等服务。
用户可以使用金融科技应用进行在线支付、理财和投资
作为一个金融科技应用的用户
我可以在应用上进行在线支付、理财和投资
同时，平台可以提供风险评估、信用评分等服务

AC 1: 用户可以在应用上进行在线支付、理财和投资
假设 用户已经登录了应用
当 用户点击在线支付、理财和投资按钮
于是 用户可以在应用上进行在线支付、理财和投资
医疗保健: 用户可以通过在线平台寻找医生、预约挂号和购买药